In [6]:
import pandas as pd

In [9]:
df = pd.read_json(path_or_buf='gg2013.json')
df.head(10)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"
3,290620657799159809,Anne Hathaway has got me living. #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': '_NicoleEdwards', 'id': 144430..."
4,290620657778188288,Jennifer Lopez's lace dress? Thoughts? #Golden...,2013-01-14 00:45:38,"{'screen_name': 'lolaogunnaike', 'id': 134953223}"
5,290620657719455745,Podrán criticar a #Adele de su moda y su maniq...,2013-01-14 00:45:38,"{'screen_name': 'SabyOnford', 'id': 404554221}"
6,290620657715273728,"US, Weakly #GoldenGlobes",2013-01-14 00:45:38,"{'screen_name': 'alejandraoleary', 'id': 45748..."
7,290620657715253248,"RT @BillMc7: ""Wait. What's that smell?!"" (ever...",2013-01-14 00:45:38,"{'screen_name': 'xtinavickers', 'id': 92139052}"
8,290620657706872832,Hugh Jackman is so awesome!!! #goldenglobes,2013-01-14 00:45:38,"{'screen_name': 'Boaz_173', 'id': 31504713}"
9,290620657560084480,It was awkward. RT @hollywoodhwife: They cut t...,2013-01-14 00:45:38,"{'screen_name': 'KateSpencer1', 'id': 21571382}"


In [11]:
data = df['text']
data[:10]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
5    Podrán criticar a #Adele de su moda y su maniq...
6                             US, Weakly #GoldenGlobes
7    RT @BillMc7: "Wait. What's that smell?!" (ever...
8          Hugh Jackman is so awesome!!! #goldenglobes
9    It was awkward. RT @hollywoodhwife: They cut t...
Name: text, dtype: object

In [12]:
data[0]

"JLo's dress! #eredcarpet #GoldenGlobes"